# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

D:\Users\admin\anaconda3\envs\transformer\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

bin D:\Users\admin\anaconda3\envs\transformer\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 7/7 [00:11<00:00,  1.67s/it]


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

## 使用微调前 ChatGLM3

In [6]:
input_text = "什么是比卦？"

In [7]:
response, history = base_model.chat(tokenizer, query=input_text)

In [8]:
print(response)

比卦是《易经》中的第六卦，由两个阴爻夹一个阳爻构成，象征着天地相摩、相互依存的關係。比卦的卦辞为：“比，吉。比，有亲亲之志，有爱民之心。”这意味着比卦象征着亲亲、和谐、和睦、团结、友爱等美好品质。

在《易经》的六十四卦中，比卦具有很高的地位。它不仅表示个人之间的亲情、友情、爱情等关系，还包含着人与自然、人与社会、人与宇宙之间的和谐相处。通过比卦的观察和分析，可以指导人们建立和谐的人际关系，促进社会进步和繁荣发展。


In [9]:
response, history = base_model.chat(tokenizer, query="比卦代表什么含义？", history=history)
print(response)

比卦在《易经》中代表和谐、和睦、团结、友爱等美好品质。它象征着天地相摩、相互依存的 relation，也代表着人物之间的亲情、友情、爱情等关系。通过比卦的观察和分析，可以指导人们建立和谐的人际关系，促进社会进步和繁荣发展。

比卦的卦辞为：“比，吉。比，有亲亲之志，有爱民之心。”这意味着比卦象征着亲亲、和谐、和睦、团结、友爱等美好品质。在比卦中，两个阴爻夹一个阳爻，象征着柔顺、温和、互相依存的关系，也代表着人们之间的互相支持和帮助。

此外，比卦还包含着人与自然、人与社会、人与宇宙之间的和谐相处，它是一种平衡的状态，使各种元素相互依存、相互促进，达到一种和谐的境界。因此，比卦在《易经》中具有非常重要的意义，它不仅是一种预测卜辞，更是一种哲学思想，指导着人们如何建立和谐关系，实现共同发展。


## 微调前后效果对比

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [11]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

### 使用微调后的 ChatGLM3-6B

In [12]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [14]:
base_response, ft_response = compare_chatglm_results(query="解释下比卦是什么？")

问题：解释下比卦是什么？

原始输出：
比卦是由两个卦相叠而成，分别是上卦坎和下卦坤。坎代表云，象征天寒而雪大，坤代表地广而平。 When the two symbols are combined, they represent a cold and snowy winter day. 

In the context of the I Ching, the比卦象征着 a time of rest and relaxation, as well as a period of等待和观望。 It is considered a favorable omen for those seeking to take action and make things happen. However, it is also recommended to be patient and wait for the right moment to act.

The比卦的卦辞描述了这一点，写道：“比， throttle the wind to get the wind to stop. 比，同声相应也。” This suggests that the best way to deal with the cold and snow is to be patient and wait for the right moment to take action.

In terms of the I Ching, the比卦 is associated with the 12th line, which represents the idea of waiting for the right moment to act. It is believed that the 12th line represents the most favorable time for any action to be taken, and that it is necessary to wait for the 12th line to appear in a divination procedure.

Overall, the比卦 in the I Ching is a symbol of patience and w

In [15]:
base_response, ft_response = compare_chatglm_results(query="地水师卦")

问题：地水师卦

原始输出：
{'name': '卜卦结果为地水师卦,说明预测的吉凶祸福如下:', 'content': '\n1. 大吉：地水师卦是一个大吉的卦象，预示着吉祥如意，事业顺利，生活平安。然而，务必小心谨慎，谨慎行事，不可轻举妄动。\n\n2. 大的吉利：地水师卦是一个大的吉利，预示着大的吉利。得到贵人的帮助，事业顺利，生活平安。\n\n3. 顺利：地水师卦预示着顺利，说明事业顺利，情况稳定。务必坚定信心，坚定决心，积极行动，可取得成功。\n\n4. 忧愁：地水师卦虽然吉利，但也要注意忧愁。必须小心谨慎，警惕风险，不可轻举妄动。\n\n5. 风险重重：地水师卦是一个凶卦，预示着风险重重，困难重重。务必小心谨慎，警惕风险，不可轻举妄动。\n\n地水师卦是一个吉祥的卦象，但也要注意忧愁。务必小心谨慎，警惕风险，不可轻举妄动。'}


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦象，下卦为坤，上卦为乾，乾为天，坤为地。地水师卦是一个由卦象组成的牌阵，它由下卦坤和上卦乾组成，代表着天地之间的相交和相互影响。在这个卦象中，天象和地象相互交通，形成地水相融的局面。此卦预示着君子观者，应当像天地相交一样，努力消除矛盾，谋求和解，然後才能获得美好的结果。

地水师卦的卦辞为：“天行地止，刚柔相济，宜顺其自然，切勿强求，方能必利。”

此卦的运势解析为：地水师卦是一个凶卦，适宜于谋求和解，消除矛盾。君子应当像天地相交一样，努力消除矛盾，寻求和解，然後才能获得美好的结果。

地水师卦的运势解析为：此卦者，事业经商皆不利。尤其是经商，容易损失财产。然而，若能坚持正德，真诚待人，尚可获得一定的收益。做事务必谨慎小心，避免 risk，等待时机，方能解救。

地水师卦的运势解析为：在感情方面，有新的发展，但需谨慎处理，避免引起冲突。经商方面，务必谨慎行事，避免风险，等待时机，方能解救。整体而言，此卦提示需坚持正德，真诚待人，等待时机，方能解救。


In [16]:
base_response, ft_response = compare_chatglm_results(query="天水讼卦")

问题：天水讼卦

原始输出：
{'name': '卜卦的结果是兑为泽（兑宫），宫位在西南方。泽上有兑，兑宫中有一卦，名为"泽泽之泽"。这是一卦动卦，代表着动不动摇的变动，动而通顺。预示着变动中raw（ raw）顺利，但也要谨慎行事。', 'content': '\n raw（raw）是一个中文词汇，表示生的、未经提炼的，引申为未经发育、未经成长。在这个卦象中，raw（raw）表示着变动的起点，也是变动的原始动力。\n\n泽泽之泽卦象中， raw（raw）卦動动起，形成宫位在西南方。这一卦象预示着动而通顺，但也要谨慎行事。\n\n在事业方面， raw（raw）卦动起，预示着事业动而通顺，但也要谨慎行事。对于经商，raw（raw）卦动起，预示着商场的变动中，raw（raw） original tendency，需要谨慎处理商业关系。对于婚恋，raw（raw）卦动起，表示爱情成长的开始。\n\n raw（raw）卦动起，预示着动而通顺，但也要谨慎行事。'}


ChatGLM3-6B微调后：
[gMASK]sop 天水讼卦卦象卦象， 君子观者，设若求理，其初若往若来，后若正立。

天雷卦：主卦为雷，动卦为雷。雷动卦中，雷声震震，君子观者，应时行义，乘时前进，可得正果。

雷卦卦象：雷声震震，天雷卦也。此卦象预示着前方有险，难以行进，但可得救解。君子应时行义，乘时前进，可以得正果。

雷卦：雷声震震，动卦为雷。此卦象预示着难以行进，但有救解。君子应时行义，乘时前进，可以得正果。

雷卦：雷声震震，静卦为雷。此卦象预示着难以行进，但有救解。君子应时行义，乘时前进，可以得正果。

雷卦：雷声震震，动卦为雷。此卦象预示着难以行进，但有救解。君子应时行义，乘时前进，可以得正果。

雷卦：雷声震震，静卦为雷。此卦象预示着难以行进，但有救解。君子应时行义，乘时前进，可以得正果。


#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [17]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [18]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B (Epoch=3, automade-dataset) 微调后：\n{ft_response}")
    return base_response, ft_response

In [19]:
base_response, ft_response = compare_chatglm_results(query="比卦的核心理念是什么？")

问题：比卦的核心理念是什么？

原始输出：
比卦是周易中的一卦，由两个卦相比较而得。它是UP（卦象）和GP（卦象）两个卦象叠加而成，代表联结紧固、互相支撑的情况。比卦的核心哲学是： Compare, Connect, and United. 比卦预示着顺和、安泰，但也警惕着可能产生不义之财，因此，君子观此卦象，必须谨慎行事，必须团结Original Thinker（原思考者）的想法，才能顺利行事。比卦的核心运势是：工作、经商、求名、婚恋等各种事业都会成功，但需警惕不义之财。对于经商，必须强调互惠互利、合作共赢的原则，同时要警惕金钱的诱惑。对于求名，需努力拼搏、勤奋学习，才能取得佳成绩。对于婚恋，需真诚相待、互相尊重，但也需注意分寸。比卦的核心哲学是： compare（比较）、connect（联系）和united（联合）。它强调的是互相关心、互帮互助、互相提携，形成一个良好的互动氛围，从而获得顺利的结果。


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 比卦的核心理念是什么？ 在周易中，比卦代表着相亲相依的意涵，它是由上卦坎（水）和下卦坤（地）相叠而成。这一卦象预示着长期的吉利和无咎，同时也暗示着不愿臣服的邦国迟迟不来朝会有难。

比卦的核心哲学是：水附大地，地纳河海，象征相亲相依，亲密无间，展示出宽宏无私，精诚团结的道理。因此，在这一卦象中，人们可以得到贵人的提拔，事业可望成功，但需诚实、信任地做事，待人宽厚、正直，主动热情。在经商中，也需真诚交往，遵守商业道德，不可贪心不足或自以为是。

比卦的运势平顺，事业顺利可望成功，可得贵人提拔。经商方面，愿望能够实现且有利润，但需与他人密切合作，讲究商业道德。在婚恋方面，象征着美好姻缘和相亲相爱。在决策中，建议心地善良，待人忠诚、厚道，工作勤恳并善于选择朋友。


In [20]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
占卜是一种古老的预测 Future 的方法，通过预测来了解自己的命运和周围环境。地水师卦是一个由三个卦相组合而成的卦象，代表着地球、水和军队三种力量。在占卜中，的解释取决于卦象的组合和卦辞。地水师卦在卜问中往往预示着困难和挫折，但也有化解困难的机会。君子观此卦象，预示着只要发展趋势正确，的解释就提示着flexibility and adaptability in the face of adversity。地水师卦是一个占卜中预示着困难的卦象，但也有化解困难的机会。


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由三个卦相组成的卦象，分别是地卦（下卦）、水卦（中间卦）和雷卦（上卦）。这个卦象预示着雷雨天气，雷声震耳，雷电交加，天气阴沉。在这个卦象中，雷卦代表天雷，地卦代表坤卦，水卦代表兑卦。兑卦代表兑宫，是兑卦初成，预示着开始兑泽。兑宫是一个吉利的宫象，但兑卦本身却是一个凶卦，因为兑卦的主卦是兑卦，而兑卦的初成却是一阳初生。因此，兑宫预示着初吉终凶，需要警惕和谨慎。

地水师卦的核心哲学是：雷雨天气，雷声震耳，雷电交加，天气阴沉。然而，雷卦代表天雷，地卦代表坤卦，兑卦代表雷卦，它们相互交织，形成一个卦象。在这种情况下，雷卦代表天雷，地卦代表坤卦，兑卦代表雷卦，它们相互交织，形成一个卦象。

地水师卦的核心哲学是：雷雨天气，雷声震耳，雷电交加，天气阴沉。然而，雷卦代表天雷，地卦代表坤卦，兑卦代表雷卦，它们相互交织，形成一个卦象。

地水师卦的核心哲学是：雷雨天气，雷声震耳，雷电交加，天气阴沉。然而，雷卦代表天雷，地卦代表坤卦，兑卦代表雷卦，它们相互交织，形成一个卦象。

地水师卦的核心哲学是：雷雨天气，雷声震耳，雷电交加，天气阴沉。然而，雷卦代表天雷，地卦代表坤卦，兑卦代表雷卦，它们相互交织，形成一个卦象。

地水师卦的核心哲学是：雷雨天气，雷声震耳，雷电交加，天气阴沉。然而，雷卦代表天雷，地卦代表坤卦，兑卦代表雷卦，它们相互交织，形成一个卦象。

地水师卦的核心哲学是：雷雨天气，雷声震耳，雷电交加，天气阴沉。然而，雷卦代表天雷，地卦代表坤卦，兑卦代表雷卦，它们


In [22]:
base_response, ft_response = compare_chatglm_results(query="周易中的乾卦讲述了什么？")

问题：周易中的乾卦讲述了什么？

原始输出：
其在下卦中自养，在上卦中养活别人，这就是乾卦的意义。乾卦的卦象是由六个阳爻组成的直角三角形，象征着天。它预示着天圆寂后，阳生阴死，原始的道德秩序被破坏，然而在此之后，天会再次出现，光明大道得以再生。在占卜中，乾卦代表亨通，但也有 risk of loss。乾卦的核心哲学是：亨通需要阳生阴死，顺应自然，发展和提高。在事业上，乾卦预示着亨通，但需要刚毅和决断力。在经商中，乾卦表示充满机遇，但 also需要有决断力和勇气。在求名中，乾卦表示充满机遇，但需要决断力和勇气。在婚恋中，乾卦表示男性充满活力，女性充满激情。在决策中，乾卦表示陽生阴死，需要顺应自然，发挥自己的潜力和智慧。


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 周易中的乾卦讲述了什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。
